For CPAIOR 2023

In [1]:
import pandas as pd
import numpy as np
from statistics import geometric_mean

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [2]:
# import result sets
df_ml = pd.read_csv("ml.csv")
df_ml = df_ml.drop(['Unnamed: 0', 'Time Label', 'Nodes Label', 'GAP W/ CUTS', 
                   'GAP W/O CUTS'], axis = 1)
df_ml.rename(columns={'INITIAL LP':'INITIAL LP W/O CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/O CUTS'}, inplace=True)
df = pd.read_csv("results_1_new.csv")

analysis_ml = df_ml.merge(df, on = ['NAME', 'RANDOM SEED'], how = 'left')

analysis_ml = analysis_ml.drop(['BEST SOLUTION', 'BEST PRIMAL BOUND', 'GAP', 'No. OF CUTS GENERATED', 'No. OF CUTS APPLIED',
                                'No. OF CUTS PRESENT IN LP AT THE END', 'No. OF NODES', 'RANDOM SEED', 'CUT SETTING',
                                'STATUS', 'END', 'SOLUTION TIME'], axis = 1)

analysis_ml.rename(columns={'INITIAL LP':'INITIAL LP W/ CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/ CUTS'}, inplace=True)

analysis_ml['CUTS OFF_FIRST ROUND'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME FIRST ROUND']
analysis_ml['CUTS OFF_ROOT END'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME ROOT END']

analysis_ml = analysis_ml.drop(['TIME ROOT END', 'ROUND OF CUTS', 'OBJECTIVE VALUE ROOT NODE', 'TIME W/O CUTS',
                                'CUTS OFF_ROOT END', 'INITIAL LP TIME'], axis = 1)

analysis_ml = analysis_ml[['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'PRESOLVING TIME W/ CUTS', 'PRESOLVING TIME W/O CUTS',
                            'TIME FIRST ROUND','NODES W/ CUTS', 'NODES W/O CUTS', 'INITIAL LP W/ CUTS', 
                           'INITIAL LP W/O CUTS', 'FIRST ROUND CUT', 'vars', 'bin_vars', 'int_vars', 'cont_vars', 
                           'constr', 'linsetpart_constr', 'linsetpack_constr', 'linsetcov_constr', 'lincard_constr', 
                           'lineqknap_constr', 'linbinpack_constr', 'linknaps_constr', 'linintknap_constr', 'linmixbin_constr',
                           'nonzeroes', 'obj_dyn', 'dynamism_mean', 'presol_vars','presol_int_vars', 'presol_constr']]

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,11.088517,13.639123,0.067284,0.062201,1.426593,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
1,10teams.mps.gz,11.186637,13.822576,0.057992,0.060087,1.378128,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
2,10teams.mps.gz,11.044666,13.681984,0.055842,0.058780,1.350495,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
3,10teams.mps.gz,10.835616,13.761031,0.052585,0.059116,1.293845,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
4,10teams.mps.gz,10.833424,13.779478,0.053544,0.058801,1.301532,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,wachplan.mps.gz,1427.930347,1804.294142,1.165874,1.552340,2.689512,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
2126,wachplan.mps.gz,1444.662760,1807.378575,1.031269,1.914422,2.536143,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
2127,wachplan.mps.gz,1431.659624,1739.747262,1.088057,1.507830,2.595248,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
2128,wachplan.mps.gz,1433.459125,1761.237671,1.125112,1.547767,2.649715,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025


In [5]:
analysis_ml = analysis_ml.groupby('NAME').mean()
analysis_ml = analysis_ml.reset_index()
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,10.997772,13.736838,0.057449,0.059797,1.350119,30.0,276.0,2.000000e+01,2.000000e+01,9.170000e+02,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108
1,22433.mps.gz,2.535930,0.823464,0.046082,0.025466,0.172883,1.0,5.0,2.124053e+04,2.124053e+04,2.127800e+04,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267
2,23588.mps.gz,5.527370,2.540436,0.027812,0.031946,0.140688,576.0,981.0,7.649866e+03,7.649866e+03,7.660000e+03,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981
3,30_70_45_05_100.mps.gz,282.653795,258.647469,1.986556,1.625184,18.567687,46.0,201.0,-5.336000e+03,-5.336000e+03,8.500000e+00,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654
4,30_70_45_095_100.mps.gz,743.997969,75.233190,1.994498,2.143665,19.518728,201.0,1.0,-6.132000e+03,-6.132000e+03,3.000000e+00,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,uct-subprob.mps.gz,3212.414116,18000.370434,0.121225,0.167001,0.360338,75300.0,2159025.8,-2.000000e+00,-2.000000e+00,2.440000e+02,7.721349,0.167996,0.000000,0.832004,7.587311,0.000000,0.003041,0.000000,0.000000,0.0,0.0,0.0,0.0,0.874303,0.002280,0.90309,0.000000,7.703459,0.000000,7.541683
422,umts.mps.gz,1079.840992,11605.745144,0.179507,0.230250,0.892776,144227.0,3106370.0,1.460426e+06,1.460426e+06,2.912957e+07,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152
423,unitcal_7.mps.gz,940.799926,10628.686922,6.937303,6.065447,64.953906,705.0,196989.0,1.028415e+06,1.028415e+06,1.946934e+07,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349
424,var-smallemery-m6j6.mps.gz,7171.038360,10325.784605,8.873553,8.342061,79.559820,685889.0,836578.0,-1.697219e+03,-1.697219e+03,-1.549250e+02,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321


In [6]:
time_label_1 = []

for index, row in analysis_ml.iterrows():
    if row['TIME W/ CUTS'] > row['CUTS OFF_FIRST ROUND']:
        time_label_1.append(0)
    elif row['TIME W/ CUTS'] <= row['CUTS OFF_FIRST ROUND']:
        time_label_1.append(1)
        
analysis_ml['Time Label'] = time_label_1

In [7]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,10teams.mps.gz,10.997772,13.736838,0.057449,0.059797,1.350119,30.0,276.0,2.000000e+01,2.000000e+01,9.170000e+02,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108,1
1,22433.mps.gz,2.535930,0.823464,0.046082,0.025466,0.172883,1.0,5.0,2.124053e+04,2.124053e+04,2.127800e+04,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267,0
2,23588.mps.gz,5.527370,2.540436,0.027812,0.031946,0.140688,576.0,981.0,7.649866e+03,7.649866e+03,7.660000e+03,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981,0
3,30_70_45_05_100.mps.gz,282.653795,258.647469,1.986556,1.625184,18.567687,46.0,201.0,-5.336000e+03,-5.336000e+03,8.500000e+00,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654,0
4,30_70_45_095_100.mps.gz,743.997969,75.233190,1.994498,2.143665,19.518728,201.0,1.0,-6.132000e+03,-6.132000e+03,3.000000e+00,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,uct-subprob.mps.gz,3212.414116,18000.370434,0.121225,0.167001,0.360338,75300.0,2159025.8,-2.000000e+00,-2.000000e+00,2.440000e+02,7.721349,0.167996,0.000000,0.832004,7.587311,0.000000,0.003041,0.000000,0.000000,0.0,0.0,0.0,0.0,0.874303,0.002280,0.90309,0.000000,7.703459,0.000000,7.541683,1
422,umts.mps.gz,1079.840992,11605.745144,0.179507,0.230250,0.892776,144227.0,3106370.0,1.460426e+06,1.460426e+06,2.912957e+07,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152,1
423,unitcal_7.mps.gz,940.799926,10628.686922,6.937303,6.065447,64.953906,705.0,196989.0,1.028415e+06,1.028415e+06,1.946934e+07,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349,1
424,var-smallemery-m6j6.mps.gz,7171.038360,10325.784605,8.873553,8.342061,79.559820,685889.0,836578.0,-1.697219e+03,-1.697219e+03,-1.549250e+02,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321,1


In [8]:
X = analysis_ml.iloc[0:, 0:-1]
y = analysis_ml.iloc[0:, -1]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                       'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
cols = X_train.copy()
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
cols

(340, 26) (340,)
(86, 26) (86,)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
255,0.045498,0.017984,0.027525,8.877000e+03,8.877000e+03,8877.000000,5.278115,0.933673,0.000000,0.066327,5.214936,0.152174,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000,5.214936
313,16.042202,22.825316,16.406966,-4.390700e+04,-4.390700e+04,-3719.000000,10.053114,0.999828,0.000172,0.000000,10.743415,0.000000,0.000086,0.000000,0.000259,0.0,0.0,0.04896,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995
229,68.027085,33.449583,556.933012,2.282219e-03,2.282219e-03,0.002282,11.449410,0.999361,0.000000,0.000639,12.125811,0.000336,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000,12.125811
5,1.683165,2.150924,13.898364,-5.967000e+03,-5.967000e+03,11.500000,9.304741,0.999909,0.000000,0.000091,9.431161,0.000000,0.000000,0.001524,0.000000,0.0,0.0,0.00000,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000,9.430359
425,1.102859,1.619734,2.617340,-2.400000e+01,-2.400000e+01,-9.000000,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.000000,0.144237,0.0,0.0,0.00000,0.000000,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434,6.582025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,26.047749,40.978283,0.000000,-2.000000e+05,-2.000000e+05,0.000000,10.315597,0.331126,0.000000,0.668874,10.613934,0.004914,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000,10.328690
192,7.056363,6.946319,0.000000,-3.689729e+08,-3.689729e+08,0.000000,8.464003,0.461928,0.000000,0.538072,8.095599,0.000000,0.111280,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.666768,0.036965,7.77789,1.758950,8.464003,0.000000,8.095599
117,0.039291,0.034147,0.067639,2.606000e+03,2.606000e+03,3900.698555,6.555357,0.250356,0.005690,0.743954,5.765191,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126,5.736572
47,0.028787,0.030815,0.052751,3.600000e+01,3.600000e+01,487.000000,5.446737,0.987069,0.004310,0.008621,4.077537,0.288136,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000,4.060443


In [10]:
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
cols = cols.iloc[0:, 0:]

bestfeatures = SelectKBest(score_func=f_classif, k = 'all')
fit = bestfeatures.fit(X_train,y_train)
scores = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(cols.columns)
featureScores = pd.concat([columns,scores],axis=1)
featureScores
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(26,'Score')) #print best features
data=featureScores.nlargest(26,'Score')

                    Features     Score
20                 nonzeroes  6.972330
22             dynamism_mean  3.745768
23               presol_vars  2.533280
6                       vars  2.094008
3         INITIAL LP W/ CUTS  1.635171
4        INITIAL LP W/O CUTS  1.635171
12         linsetpack_constr  1.370288
19          linmixbin_constr  0.874722
15          lineqknap_constr  0.846321
14            lincard_constr  0.773060
18         linintknap_constr  0.668918
5            FIRST ROUND CUT  0.618351
2           TIME FIRST ROUND  0.465733
0    PRESOLVING TIME W/ CUTS  0.398137
21                   obj_dyn  0.264045
24           presol_int_vars  0.102030
10                    constr  0.088921
1   PRESOLVING TIME W/O CUTS  0.081716
9                  cont_vars  0.077764
11         linsetpart_constr  0.046181
17           linknaps_constr  0.035761
8                   int_vars  0.035251
16         linbinpack_constr  0.028901
7                   bin_vars  0.019370
25             presol_con

In [12]:
X = analysis_ml.drop(['linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr',
                      'bin_vars', 'presol_constr', 'linsetcov_constr', 'Time Label'], axis = 1)
y = analysis_ml.iloc[0:, -1]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

test_df = X_test.copy()
train_df = X_train.copy()

X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 
                        'NODES W/O CUTS'], axis = 1)
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND',
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train

(340, 19) (340,)
(86, 19) (86,)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,cont_vars,constr,linsetpack_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars
255,0.045498,0.017984,0.027525,8.877000e+03,8.877000e+03,8877.000000,5.278115,0.066327,5.214936,0.000000,0.000000,0.0,0.000000,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000
313,16.042202,22.825316,16.406966,-4.390700e+04,-4.390700e+04,-3719.000000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
229,68.027085,33.449583,556.933012,2.282219e-03,2.282219e-03,0.002282,11.449410,0.000639,12.125811,0.000000,0.000000,0.0,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000
5,1.683165,2.150924,13.898364,-5.967000e+03,-5.967000e+03,11.500000,9.304741,0.000091,9.431161,0.000000,0.000000,0.0,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000
425,1.102859,1.619734,2.617340,-2.400000e+01,-2.400000e+01,-9.000000,8.119994,0.000000,7.347944,0.000000,0.144237,0.0,0.000000,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,26.047749,40.978283,0.000000,-2.000000e+05,-2.000000e+05,0.000000,10.315597,0.668874,10.613934,0.000000,0.000000,0.0,0.000000,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000
192,7.056363,6.946319,0.000000,-3.689729e+08,-3.689729e+08,0.000000,8.464003,0.538072,8.095599,0.111280,0.000000,0.0,0.000000,0.666768,0.036965,7.77789,1.758950,8.464003,0.000000
117,0.039291,0.034147,0.067639,2.606000e+03,2.606000e+03,3900.698555,6.555357,0.743954,5.765191,0.000000,0.000000,0.0,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126
47,0.028787,0.030815,0.052751,3.600000e+01,3.600000e+01,487.000000,5.446737,0.008621,4.077537,0.000000,0.000000,0.0,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000


In [14]:
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
model_params={'svm':{'model': SVC(kernel = 'rbf', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 'gamma': [1e-2, 1e-3, 1e-4]
                             }
                    },
              'rfm':{'model': RandomForestClassifier(random_state = 0), 
                    'params':{'n_estimators':[100, 200, 500], 
                             'max_depth':[5, 10, 15]
                             }
                    },
              'lrm':{'model': LogisticRegression(solver='lbfgs', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 
                              'max_iter':[100, 500, 1000]
                             }
                    },
              'et':{'model': ExtraTreesClassifier(random_state=0), 
                    'params':{'n_estimators': [100, 200, 500], 
                              'max_depth': [5, 10, 15]
                             }
                    },
              'nn':{'model': MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', 
                                           power_t=0.5, max_iter=500, shuffle=True, tol=0.00001, verbose=True, 
                                           early_stopping=False, validation_fraction=0.1, random_state=0),
                    'params':{'hidden_layer_sizes': [(5, ), (10, ), (15, )],
                                 'learning_rate_init': [1e-1, 1e-2, 1e-3],
                                 'alpha':  [1e-2, 1e-3, 1e-4],
                                 'activation': ['tanh', 'relu', 'logistic']
                             }
                    }
             }
scores=[]
for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], cv = None, return_train_score = False, n_jobs = -1)
    clf.fit(X_train, y_train)
    scores.append({'model': model_name, 'best_score':clf.best_score_, 'best_params':clf.best_params_})

C:\Users\ambar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
scores=pd.DataFrame(scores).sort_values(by = ['best_score'], ascending = False)
scores.to_csv('scores_new_1.csv')

KeyError: 'best_score'

In [16]:
scores = OrderedDict()
target_names = ['0', '1']

print('SUPPORT VECTOR MACHINES')
clf = SVC(random_state = 0, C = 1000, gamma = 0.001, kernel = 'rbf')
clf.fit(X_train, y_train)
y_train_pred_svm = clf.predict(X_train)
y_test_pred_svm = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_svm)
mse_test = mean_squared_error(y_test, y_test_pred_svm)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_svm, target_names=target_names))
scores['SVC'] = [score, metrics.precision_score(y_test, y_test_pred_svm), metrics.recall_score(y_test, y_test_pred_svm),
                      metrics.f1_score(y_test, y_test_pred_svm), mse_test, mse_train]

print('##################################')
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_nn)
mse_test = mean_squared_error(y_test, y_test_pred_nn)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_nn, target_names=target_names))
scores['NN'] = [score, metrics.precision_score(y_test, y_test_pred_nn), metrics.recall_score(y_test, y_test_pred_nn),
                      metrics.f1_score(y_test, y_test_pred_nn), mse_test, mse_train]

print('##################################')
print('DECISION TREES')
clf = ExtraTreesClassifier(random_state=0, max_depth = 5, n_estimators = 100)
clf.fit(X_train, y_train)
y_train_pred_et = clf.predict(X_train)
y_test_pred_et = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_et)
mse_test = mean_squared_error(y_test, y_test_pred_et)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_et, target_names=target_names))
scores['ET'] = [score, metrics.precision_score(y_test, y_test_pred_et), 
                         metrics.recall_score(y_test, y_test_pred_et), 
                         metrics.f1_score(y_test, y_test_pred_et), mse_test, mse_train]

print('##################################')
print('LOGISTIC REGRESSION')
clf = LogisticRegression(solver='lbfgs', random_state = 0, C = 1000, max_iter = 100)
clf.fit(X_train, y_train)
y_train_pred_lr = clf.predict(X_train)
y_test_pred_lr = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_lr)
mse_test = mean_squared_error(y_test, y_test_pred_lr)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_lr, target_names=target_names))
scores['LR'] = [score, metrics.precision_score(y_test, y_test_pred_lr), metrics.recall_score(y_test, y_test_pred_lr),
                      metrics.f1_score(y_test, y_test_pred_lr), mse_test, mse_train]

print('##################################')
print('RANDOM FOREST')
clf = RandomForestClassifier(random_state = 0, max_depth = 5, n_estimators = 200)
clf.fit(X_train, y_train)
y_train_pred_rf = clf.predict(X_train)
y_test_pred_rf = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_rf)
mse_test = mean_squared_error(y_test, y_test_pred_rf)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_rf, target_names=target_names))
scores['RF'] = [score, metrics.precision_score(y_test, y_test_pred_rf), metrics.recall_score(y_test, y_test_pred_rf),
                      metrics.f1_score(y_test, y_test_pred_rf), mse_test, mse_train]

SUPPORT VECTOR MACHINES
MSE Test: 0.38372093023255816, MSE Train: 0.3264705882352941
Accuracy score: 0.6162790697674418

              precision    recall  f1-score   support

           0       0.33      0.03      0.06        32
           1       0.63      0.96      0.76        54

    accuracy                           0.62        86
   macro avg       0.48      0.50      0.41        86
weighted avg       0.52      0.62      0.50        86

##################################
NEURAL NETWORKS
MSE Test: 0.3023255813953488, MSE Train: 0.18529411764705883
Accuracy score: 0.6976744186046512

              precision    recall  f1-score   support

           0       0.59      0.62      0.61        32
           1       0.77      0.74      0.75        54

    accuracy                           0.70        86
   macro avg       0.68      0.68      0.68        86
weighted avg       0.70      0.70      0.70        86

##################################
DECISION TREES
MSE Test: 0.383720930232558

C:\Users\ambar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


MSE Test: 0.32558139534883723, MSE Train: 0.20294117647058824
Accuracy score: 0.6744186046511628

              precision    recall  f1-score   support

           0       0.83      0.16      0.26        32
           1       0.66      0.98      0.79        54

    accuracy                           0.67        86
   macro avg       0.75      0.57      0.53        86
weighted avg       0.73      0.67      0.59        86



In [17]:
scores_df = pd.DataFrame.from_dict(scores, orient='index')
scores_df.columns=['accuracy', 'precision', 'recall', 'f1-score', 'mse test', 'mse train']
scores_df

,accuracy,precision,recall,f1-score,mse test,mse train
SVC,0.616279,0.626506,0.962963,0.759124,0.383721,0.326471
NN,0.697674,0.769231,0.740741,0.754717,0.302326,0.185294
ET,0.616279,0.626506,0.962963,0.759124,0.383721,0.335294
LR,0.639535,0.645570,0.944444,0.766917,0.360465,0.335294
RF,0.674419,0.662500,0.981481,0.791045,0.325581,0.202941


In [18]:
test_df['PLabel NN'] = y_test_pred_nn
test_df['PLabel ET'] = y_test_pred_et
test_df['PLabel RF'] = y_test_pred_rf

test_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = test_df.merge(test_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['CUTS OFF_FIRST ROUND', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,neos-780889.mps.gz,176.675456,449.053574,1.0,1.0,1,1,1,1,176.675456,1.0
1,mzzv11.mps.gz,422.340278,1525.322799,1356.0,8880.0,1,1,1,1,422.340278,1356.0
2,chromaticindex32-8.mps.gz,1.025220,3.402795,1.0,249.0,1,1,1,1,1.025220,1.0
3,neos22.mps.gz,1.452381,18000.395746,1.0,4782054.0,1,1,1,1,1.452381,1.0
4,ab67-40-100.mps.gz,300.208641,331.249310,73.0,75.0,1,1,1,1,300.208641,73.0
...,...,...,...,...,...,...,...,...,...,...,...
81,mik-250-20-75-2.mps.gz,12.884775,24.797749,427.0,90650.0,1,1,1,1,12.884775,427.0
82,30_70_45_095_100.mps.gz,743.997969,75.233190,201.0,1.0,1,1,1,0,75.233190,1.0
83,neos-4322846-ryton.mps.gz,818.066564,845.596316,664.0,522.0,1,1,1,1,818.066564,522.0
84,ran14x18-disj-8.mps.gz,1658.803667,16537.955252,312024.0,14591597.0,1,1,1,1,1658.803667,312024.0


In [19]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == 0.0:
        rf_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == 0.0:
        nn_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == 0.0:
        et_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,neos-780889.mps.gz,176.675456,449.053574,1.0,1.0,1,1,1,1,176.675456,1.0,176.675456,176.675456,176.675456,1.0,1.0,1.0
1,mzzv11.mps.gz,422.340278,1525.322799,1356.0,8880.0,1,1,1,1,422.340278,1356.0,422.340278,422.340278,422.340278,1356.0,1356.0,1356.0
2,chromaticindex32-8.mps.gz,1.025220,3.402795,1.0,249.0,1,1,1,1,1.025220,1.0,1.025220,1.025220,1.025220,1.0,1.0,1.0
3,neos22.mps.gz,1.452381,18000.395746,1.0,4782054.0,1,1,1,1,1.452381,1.0,1.452381,1.452381,1.452381,1.0,1.0,1.0
4,ab67-40-100.mps.gz,300.208641,331.249310,73.0,75.0,1,1,1,1,300.208641,73.0,300.208641,300.208641,300.208641,73.0,73.0,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,mik-250-20-75-2.mps.gz,12.884775,24.797749,427.0,90650.0,1,1,1,1,12.884775,427.0,12.884775,12.884775,12.884775,427.0,427.0,427.0
82,30_70_45_095_100.mps.gz,743.997969,75.233190,201.0,1.0,1,1,1,0,75.233190,1.0,743.997969,743.997969,743.997969,201.0,201.0,201.0
83,neos-4322846-ryton.mps.gz,818.066564,845.596316,664.0,522.0,1,1,1,1,818.066564,522.0,818.066564,818.066564,818.066564,664.0,664.0,664.0
84,ran14x18-disj-8.mps.gz,1658.803667,16537.955252,312024.0,14591597.0,1,1,1,1,1658.803667,312024.0,1658.803667,1658.803667,1658.803667,312024.0,312024.0,312024.0


In [20]:
testing = analysis.copy()

In [21]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [22]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [23]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [24]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_FIRST ROUND'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [25]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      319.066487    449.350777   230.633527   
               [10,18000)     668.496297    909.379609   449.950650   
               [100,18000)   1635.239844   1740.003555  1050.404535   
               [1000,18000)  7372.482963   5603.443218  3944.664342   
               [3600,18000)  9273.146949  11525.196682  7691.035065   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      305.229328   316.488669   316.194048   
               [10,18000)     632.440068   661.718418   656.284019   
               [100,18000)   1535.117613  1637.348840  1372.583258   
               [1000,18000)  6369.605521  7372.482963  4773.296210   
               [3600,18000)  9193.238499  9273.146949  8675.809381   

Parameter                             Node                                \
Setting                                Cut         No Cut        Perfect   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6490.303382   18481.916810    5701.479095   
               [10,18000)     13336.715273   39828.740403   11324.046990   
               [100,18000)    25332.878728   45977.245793   20701.308464   
               [1000,18000)  332913.273432  439692.731118  219698.823680   
               [3600,18000)  530800.984581  819106.252775  482884.718216   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6467.396558    6471.209822    8267.537835   
               [10,18000)     13276.897484   13224.301569   17380.449852   
               [100,18000)    25547.131558   25400.567790   27343.608811   
               [1000,18000)  337254.667203  332913.273432  321195.695240   
               [3600,18000)  529423.905324  530800.984581  646928.277282   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)           86  
               [10,18000)          63  
               [100,18000)         42  
               [1000,18000)        18  
               [3600,18000)         9

In [26]:
imp_time_nn = []
imp_nodes_nn = []

for i, j in zip(t_nn, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_nn.append(imp)

for i, j in zip(n_nn, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_nn.append(imp)

In [27]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_nn
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_nn
avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      319.066487    449.350777   230.633527   
               [10,18000)     668.496297    909.379609   449.950650   
               [100,18000)   1635.239844   1740.003555  1050.404535   
               [1000,18000)  7372.482963   5603.443218  3944.664342   
               [3600,18000)  9273.146949  11525.196682  7691.035065   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      305.229328   316.488669   316.194048   
               [10,18000)     632.440068   661.718418   656.284019   
               [100,18000)   1535.117613  1637.348840  1372.583258   
               [1000,18000)  6369.605521  7372.482963  4773.296210   
               [3600,18000)  9193.238499  9273.146949  8675.809381   

Parameter                             Node                                \
Setting                                Cut         No Cut        Perfect   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6490.303382   18481.916810    5701.479095   
               [10,18000)     13336.715273   39828.740403   11324.046990   
               [100,18000)    25332.878728   45977.245793   20701.308464   
               [1000,18000)  332913.273432  439692.731118  219698.823680   
               [3600,18000)  530800.984581  819106.252775  482884.718216   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6467.396558    6471.209822    8267.537835   
               [10,18000)     13276.897484   13224.301569   17380.449852   
               [100,18000)    25547.131558   25400.567790   27343.608811   
               [1000,18000)  337254.667203  332913.273432  321195.695240   
               [3600,18000)  529423.905324  530800.984581  646928.277282   

Parameter                   Instances Improvement (%)             
Setting                                          Time      Nodes  
Metric         Bucket                                             
Geometric Mean [0,18000)           86       -0.900263  27.382918  
               [10,18000)          63       -1.826828  30.320319  
               [100,18000)         42      -16.062267   7.937235  
               [1000,18000)        18      -35.255243  -3.519709  
               [3600,18000)         9       -6.441584  21.877746

In [28]:
train_df

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,cont_vars,constr,linsetpack_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars
255,neos-5140963-mincio.mps.gz,16296.812996,17910.845801,0.045498,0.017984,0.027525,5624805.0,5527433.2,8.877000e+03,8.877000e+03,8877.000000,5.278115,0.066327,5.214936,0.000000,0.000000,0.0,0.000000,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000
313,neos8.mps.gz,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-4.390700e+04,-4.390700e+04,-3719.000000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
229,neos-4391920-timok.mps.gz,299.384161,847.066375,68.027085,33.449583,556.933012,1.0,1.0,2.282219e-03,2.282219e-03,0.002282,11.449410,0.000639,12.125811,0.000000,0.000000,0.0,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000
5,30_70_45_095_98.mps.gz,76.557754,40.224768,1.683165,2.150924,13.898364,1.0,1.0,-5.967000e+03,-5.967000e+03,11.500000,9.304741,0.000091,9.431161,0.000000,0.000000,0.0,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000
425,wachplan.mps.gz,1434.605408,1779.156125,1.102859,1.619734,2.617340,67520.0,98056.0,-2.400000e+01,-2.400000e+01,-9.000000,8.119994,0.000000,7.347944,0.000000,0.144237,0.0,0.000000,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,ns1644855.mps.gz,860.987882,853.974045,26.047749,40.978283,0.000000,1.0,1.0,-2.000000e+05,-2.000000e+05,0.000000,10.315597,0.668874,10.613934,0.000000,0.000000,0.0,0.000000,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000
192,neos-3065804-namu.mps.gz,11.837614,11.855551,7.056363,6.946319,0.000000,1.0,1.0,-3.689729e+08,-3.689729e+08,0.000000,8.464003,0.538072,8.095599,0.111280,0.000000,0.0,0.000000,0.666768,0.036965,7.77789,1.758950,8.464003,0.000000
117,ic97_tension.mps.gz,20.023839,32.066948,0.039291,0.034147,0.067639,22278.0,83938.0,2.606000e+03,2.606000e+03,3900.698555,6.555357,0.743954,5.765191,0.000000,0.000000,0.0,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126
47,bppc8-02.mps.gz,0.677820,0.169876,0.028787,0.030815,0.052751,125.0,177.0,3.600000e+01,3.600000e+01,487.000000,5.446737,0.008621,4.077537,0.000000,0.000000,0.0,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000


In [29]:
train_df['PLabel NN'] = y_train_pred_nn
train_df['PLabel ET'] = y_train_pred_et
train_df['PLabel RF'] = y_train_pred_rf
train_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = train_df.merge(train_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['CUTS OFF_FIRST ROUND', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,neos-5140963-mincio.mps.gz,16296.812996,17910.845801,5624805.0,5527433.2,1,1,1,1,16296.812996,5527433.2
1,neos8.mps.gz,16.780323,72.890510,1.0,1.0,1,1,1,1,16.780323,1.0
2,neos-4391920-timok.mps.gz,299.384161,847.066375,1.0,1.0,1,1,1,1,299.384161,1.0
3,30_70_45_095_98.mps.gz,76.557754,40.224768,1.0,1.0,0,1,1,0,40.224768,1.0
4,wachplan.mps.gz,1434.605408,1779.156125,67520.0,98056.0,1,1,1,1,1434.605408,67520.0
...,...,...,...,...,...,...,...,...,...,...,...
335,ns1644855.mps.gz,860.987882,853.974045,1.0,1.0,0,1,0,0,853.974045,1.0
336,neos-3065804-namu.mps.gz,11.837614,11.855551,1.0,1.0,1,1,1,1,11.837614,1.0
337,ic97_tension.mps.gz,20.023839,32.066948,22278.0,83938.0,1,1,1,1,20.023839,22278.0
338,bppc8-02.mps.gz,0.677820,0.169876,125.0,177.0,0,0,0,0,0.169876,125.0


In [30]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == 0.0:
        rf_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == 0.0:
        nn_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == 0.0:
        et_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,neos-5140963-mincio.mps.gz,16296.812996,17910.845801,5624805.0,5527433.2,1,1,1,1,16296.812996,5527433.2,16296.812996,16296.812996,16296.812996,5624805.0,5624805.0,5624805.0
1,neos8.mps.gz,16.780323,72.890510,1.0,1.0,1,1,1,1,16.780323,1.0,16.780323,16.780323,16.780323,1.0,1.0,1.0
2,neos-4391920-timok.mps.gz,299.384161,847.066375,1.0,1.0,1,1,1,1,299.384161,1.0,299.384161,299.384161,299.384161,1.0,1.0,1.0
3,30_70_45_095_98.mps.gz,76.557754,40.224768,1.0,1.0,0,1,1,0,40.224768,1.0,76.557754,40.224768,76.557754,1.0,1.0,1.0
4,wachplan.mps.gz,1434.605408,1779.156125,67520.0,98056.0,1,1,1,1,1434.605408,67520.0,1434.605408,1434.605408,1434.605408,67520.0,67520.0,67520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,ns1644855.mps.gz,860.987882,853.974045,1.0,1.0,0,1,0,0,853.974045,1.0,853.974045,853.974045,860.987882,1.0,1.0,1.0
336,neos-3065804-namu.mps.gz,11.837614,11.855551,1.0,1.0,1,1,1,1,11.837614,1.0,11.837614,11.837614,11.837614,1.0,1.0,1.0
337,ic97_tension.mps.gz,20.023839,32.066948,22278.0,83938.0,1,1,1,1,20.023839,22278.0,20.023839,20.023839,20.023839,22278.0,22278.0,22278.0
338,bppc8-02.mps.gz,0.677820,0.169876,125.0,177.0,0,0,0,0,0.169876,125.0,0.169876,0.169876,0.169876,177.0,177.0,177.0


In [31]:
training = analysis.copy()

In [32]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [33]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [34]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [35]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_FIRST ROUND'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [36]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                            Time                             \
Setting                               Cut        No Cut      Perfect   
Metric         Bucket                                                  
Geometric Mean [0,18000)       350.004190    564.268053   288.083546   
               [10,18000)      599.675183    959.213894   478.533071   
               [100,18000)    1513.164666   2369.664484  1185.191913   
               [1000,18000)   5641.902073   7399.674760  4258.732783   
               [3600,18000)  11438.455128  14022.952086  9726.471911   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)       324.265832    339.686747    319.476045   
               [10,18000)      548.535409    578.935294    538.628371   
               [100,18000)    1375.397692   1477.850148   1339.571625   
               [1000,18000)   5111.189126   5490.342933   4767.501932   
               [3600,18000)  10170.235095  11047.113602  10385.267617   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      4096.962302   13334.591257   3569.851283   
               [10,18000)     6523.995528   20540.982619   5565.633702   
               [100,18000)   12238.072667   36143.911429  10292.610823   
               [1000,18000)  32547.646738   84198.795424  25537.745334   
               [3600,18000)  47490.851527  123838.557457  37048.522613   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      3978.576767   3992.198359   4555.655469   
               [10,18000)     6301.786037   6329.258291   7295.953708   
               [100,18000)   12029.094882  12069.878996  14346.799433   
               [1000,18000)  33457.138355  32133.534166  40281.232439   
               [3600,18000)  54233.698961  46647.092286  69374.129371   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)          340  
               [10,18000)         271  
               [100,18000)        183  
               [1000,18000)        90  
               [3600,18000)        46

In [37]:
imp_time_nn = []
imp_nodes_nn = []

for i, j in zip(t_nn, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_nn.append(imp)

for i, j in zip(n_nn, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_nn.append(imp)

In [38]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_nn
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_nn
avg_bb_df

Parameter                            Time                             \
Setting                               Cut        No Cut      Perfect   
Metric         Bucket                                                  
Geometric Mean [0,18000)       350.004190    564.268053   288.083546   
               [10,18000)      599.675183    959.213894   478.533071   
               [100,18000)    1513.164666   2369.664484  1185.191913   
               [1000,18000)   5641.902073   7399.674760  4258.732783   
               [3600,18000)  11438.455128  14022.952086  9726.471911   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)       324.265832    339.686747    319.476045   
               [10,18000)      548.535409    578.935294    538.628371   
               [100,18000)    1375.397692   1477.850148   1339.571625   
               [1000,18000)   5111.189126   5490.342933   4767.501932   
               [3600,18000)  10170.235095  11047.113602  10385.267617   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      4096.962302   13334.591257   3569.851283   
               [10,18000)     6523.995528   20540.982619   5565.633702   
               [100,18000)   12238.072667   36143.911429  10292.610823   
               [1000,18000)  32547.646738   84198.795424  25537.745334   
               [3600,18000)  47490.851527  123838.557457  37048.522613   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      3978.576767   3992.198359   4555.655469   
               [10,18000)     6301.786037   6329.258291   7295.953708   
               [100,18000)   12029.094882  12069.878996  14346.799433   
               [1000,18000)  33457.138355  32133.534166  40281.232439   
               [3600,18000)  54233.698961  46647.092286  69374.129371   

Parameter                   Instances Improvement (%)             
Setting                                          Time      Nodes  
Metric         Bucket                                             
Geometric Mean [0,18000)          340       -8.722223  11.195933  
               [10,18000)         271      -10.179980  11.832598  
               [100,18000)        183      -11.472184  17.230873  
               [1000,18000)        90      -15.498322  23.760814  
               [3600,18000)        46       -9.207428  46.078933

In [39]:
df = [training, testing]

t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_FIRST ROUND'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [40]:
train_test_classes = ['Geometric Mean']
train_test_sets = ['Train Set', 'Test Set']

In [41]:
train_test_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut', 'No Cut','Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

train_test_row_names = pd.MultiIndex.from_product(
    [train_test_classes, train_test_sets],
    names=['Metric', 'Set'])

train_test_df = pd.DataFrame(
    index = train_test_row_names,
    dtype = float)

train_test_df = pd.DataFrame(
    columns = train_test_cols,
    index = train_test_row_names,
    dtype = float)

In [42]:
train_test_df['Instances'] = instances

train_test_df[('Time', 'Cut')] = t_cuts
train_test_df[('Time', 'No Cut')] = t_nocuts
train_test_df[('Time', 'RF')] = t_rf
train_test_df[('Time', 'ET')] = t_et
train_test_df[('Time', 'NN')] = t_nn

train_test_df[('Time', 'Perfect')] = t_perfect

train_test_df[('Node', 'Cut')] = n_cuts
train_test_df[('Node', 'No Cut')] = n_nocuts
train_test_df[('Node', 'RF')] = n_rf
train_test_df[('Node', 'ET')] = n_et
train_test_df[('Node', 'NN')] = n_nn

train_test_df[('Node', 'Perfect')] = n_perfect

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  350.004190  564.268053  288.083546  324.265832   
               Test Set   319.066487  449.350777  230.633527  305.229328   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  339.686747  319.476045  4096.962302  13334.591257   
               Test Set   316.488669  316.194048  6490.303382  18481.916810   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  3569.851283  3978.576767  3992.198359  4555.655469   
               Test Set   5701.479095  6467.396558  6471.209822  8267.537835   

Parameter                Instances  
Setting                             
Metric         Set                  
Geometric Mean Train Set       340  
               Test Set         86

In [43]:
best_time = []
best_nodes = []

zip_time = zip(t_rf, t_et, t_nn)
zip_time = list(zip_time)

zip_nodes = zip(n_rf, n_et, n_nn)
zip_nodes = list(zip_nodes)

for i in zip_time:
    best_time.append(min(i))

for i in zip_nodes:
    best_nodes.append(min(i))

In [44]:
imp_time = []
imp_nodes = []

for i, j in zip(best_time, t_cuts):
    imp = ((i - j)/j) * 100
    imp_time.append(imp)
    
for i, j in zip(best_nodes, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes.append(imp)

In [45]:
train_test_df[('Improvement (%)', 'Time')] = imp_time
train_test_df[('Improvement (%)', 'Nodes')] = imp_nodes

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  350.004190  564.268053  288.083546  324.265832   
               Test Set   319.066487  449.350777  230.633527  305.229328   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  339.686747  319.476045  4096.962302  13334.591257   
               Test Set   316.488669  316.194048  6490.303382  18481.916810   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  3569.851283  3978.576767  3992.198359  4555.655469   
               Test Set   5701.479095  6467.396558  6471.209822  8267.537835   

Parameter                Instances Improvement (%)            
Setting                                       Time     Nodes  
Metric         Set                                            
Geometric Mean Train Set       340       -8.722223 -2.889593  
               Test Set         86       -4.336764 -0.352939

In [50]:
for_plot = pd.concat([training, testing])
for_plot = for_plot.reset_index()
for_plot.to_csv('first_round_plot.csv')